Dieses Jupyter Notebook ist eine Ergänzung zu der Bachelorarbeit
<h2><center> Lösen des Minimierungsproblems: Es existiert kein 16-Tipp Sudoku </center></h2>
von Jonas Widmann an der Universität Stuttgart. Wir wollen dabei nun die in der Ausarbeitung gelernten Methoden benutzen, um mit diesen das Minimierungsproblem für Shidokus explizit zu betrachten. Shidoku sind Mini-Sudoku mit einem $4\times4$-Gitter. Wir betrachten also besonders die Aufgabenstellung:
<h2><center> Lösen des Minimierungsproblems: Es existiert kein 3-Tipp Shidoku </center></h2>
Der folgende Code wird zwar jeweils kurz erklärt, jedoch ist dieser nicht alleine ohne die Bachelorarbeit zu betrachten. Das liegt daran, dass Überlegungen und Hilfsaussagen der benutzen Methoden und Aussagen nicht ausführlich aufgezeigt werden.

In [ ]:
import numpy as np
import itertools

Wir wollen nun also das Minimierungsproblem für Shidokus betrachten. Die Spielregeln für Shudokus sind dabei: 
1. In jeder Reihe müssen die Zahlen von $1$ bis $4$ genau ein Mal vorkommen.
2. In jeder Spalte müssen die Zahlen von $1$ bis $4$ genau ein Mal vorkommen.
3. In jedem Block müssen die Zahlen von $1$ bis $4$ genau ein Mal vorkommen.

Ein Sudoku-Rätsel muss nun in Anbetracht der Spielregeln mit den Zahlen $1$ bis $4$ ausgefüllt werden, bis jede Zelle in dem Gitter einen Wert zugeordnet hat, das Gitter also vollständig ist. Wir wollen nun einen Shidoku-Solver programmieren, welcher ein Shidoku-Rätsel numerisch lösen kann. Dafür benutögen wir folgende (Hilfs-)Funktionen:

In [ ]:
def number_valid(board, number, position): 
    '''
    This function checks whether a given number is valid for the given position in the shidoku-board.
    The number is valid, if none of the associated cells of the cell of the given position already contain the number.
    The number is not valid, if a associated cells containing the number exists, otherwise the rules would be broken.
    '''
    
    # check if row contains the number
    for i in range(len(board[0])):
        if board[position[0]][i] == number and position[1] != i:
            return False
        
    # check if column contains the number
    for i in range(len(board)):
        if board[i][position[1]] == number and position[0] != i:
            return False
        
    # check if block contains the number
    block_x = position[1] // 2
    block_y = position[0] // 2
    for i in range(block_y * 2, block_y * 2 + 2):
        for j in range(block_x * 2, block_x * 2 + 2):
            if board[i][j] == number and (i,j) != number:
                return False
            
    return True


def solve(board, solutions):
    '''
    This function recursively fills a given shidoku-board with all possibilities,
    using backtracking if no number is valid in the cell. 
    If a solution is found, it is then added to the list of the solutions of the board, allowing us to calculate all solution
    grids of the given Shidoku-board.
    '''
    
    for y in range(len(board)):
        for x in range(len(board)):
            if board[y][x] == 0: # cell is empty
                for n in range(1,len(board)+1):
                    if number_valid(board, n, (y,x)):
                        board[y][x] = n # number is valid
                        solve(board, solutions)
                        board[y][x] = 0 # backtrack if the recursive call gets no valid number for the following cell.
                        
                return
    solutions.append(board.copy())
    
        
def find_solutions(board):
    '''
    This function takes a Shidoku-board, creates an empty list for the solution and than calls the function solve(),
    to get all the solutions of the board.
    '''
    
    solutions = []
    solve(board, solutions)
    return solutions

Diesen Solver können wir beispielsweise benutzen, um die in dem Paper berechnete Anzahl an verschiedenen Shidoku-Gittern zu berechnen. Wir waren in dem Paper auf $288$ verschiedene Shidoku-Gitter bekommen. Dies müsste die gleiche Anzahl an Gittern sein, die der eben definierte Solver als Lösungen des leeren Shidoku-Gitters finden sollte.

In [ ]:
empty_shidoku = np.zeros([4,4]) # empty Shidoku-Grid
shidoku_grids = find_solutions(empty_shidoku)
print("Die Anzahl an verschiedenen Shidoku-Gittern ist:", len(shidoku_grids))

Dies bestätigt also, dass $288$ verschiedene Shidoku-Gitter existieren.\
Wir wollen nun die Äquivalenztransformationen auf Shidoku-Gittern betrachten. Wir hatten hier gesagt, dass die Äquivalenztransformationen sind:
1. Zahlenpermutation: Umnummerierung der Zahlen
2. Reihenpermutation: Permutation der Bänder und der Reihen in dem jeweiligen Band
3. Spaltenpermutation: Permutation der Stapel und der Spalten in dem jeweiligen Stapel
4. Transposition des Gitters

Diese Äquivalenztransformationen können wir direkt definieren.

In [ ]:
# all permutations in symmetrical group S_4
number_permutations = list(itertools.permutations([1,2,3,4]))

# all permutations of the bands and the rows in each of the bands
row_permutations = ([0,1,2,3],[0,1,3,2],[1,0,2,3],[1,0,3,2],[2,3,0,1],[3,2,0,1],[2,3,1,0],[3,2,1,0])

# all permutations of the stacks and the columns in each of the stacks
column_permutations = ([0,1,2,3],[0,1,3,2],[1,0,2,3],[1,0,3,2],[2,3,0,1],[3,2,0,1],[2,3,1,0],[3,2,1,0])

# either the transpose is tacken, or not
transpose = (False,True)

Für diese Äquivalenztransformationen wollen wir nun jeweils die Auswirkung auf ein Shidoku-Gitter numerisch berechnen. Diese Funktionen sind technisch nicht sonderlich aufwendig, sie machen genau das, was sie logisch machen sollten.

In [ ]:
def permutate_numbers(grid, permutation=[0,1,2,3]):
    '''
    This function permutates the numbers in the given grid with the given number permutation.
    '''
    
    permutated_grid = grid.copy()
    #np.select substitutes specific values in an array with new values
    permutated_grid = np.select([permutated_grid == 1, permutated_grid == 2, permutated_grid == 3, permutated_grid == 4],
                                permutation, permutated_grid)
    return permutated_grid


def permutate_rows(grid, permutation=[0,1,2,3]):
    '''
    This function permutates the rows in the given grid with the given row permutation.
    '''
    
    permutated_grid = grid.copy()
    permutated_grid[permutation] = permutated_grid[[0,1,2,3]]
    return permutated_grid


def permutate_columns(grid, permutation=[0,1,2,3]):
    '''
    This function permutates the columns in the given grid with the given column permutation.
    '''
    
    permutated_grid = grid.copy()
    permutated_grid[:,permutation] = permutated_grid[:,[0,1,2,3]]
    return permutated_grid


def transpose_grid(grid, transpose = False):
    '''
    This functions transposes the grid if the given variable transpose is True.
    '''
    
    if transpose:
        grid = np.transpose(grid).copy()
    return grid

Mit den eben definierten Funktionen können wir nun mit einem Brute-Force Algorithmus die komplette Äquivalenzklasse zu einem Gitter bestimmen. Dafür erzeugen wir alle Gitter, die mit den Äquivalenztransformationen möglich sind und fügen diese einer Listen hinzu, falls das erzeugte Gitter nicht bereits in dieser Liste vorhanden ist.

In [ ]:
def equivalent_grids(grid):
    '''
    This function calculates all equivalent grids to a given grid using all possible combinations
    of the equivalence transformations.
    '''
    
    equiv_grids = []
    for i in transpose:
        for j in row_permutations:
            for k in column_permutations:
                for l in number_permutations:
                    equiv_grid = permutate_numbers(permutate_columns(permutate_rows(transpose_grid(grid,i),j),k),l)
                    
                    if not any(np.array_equal(equiv_grid, equi) for equi in equiv_grids): # check if grid is already in the list
                        equiv_grids.append(equiv_grid)
    return equiv_grids

Wir hatten in der Ausarbeitung gesehen, dass es genau zwei Äquivalenzklassen für Shidoku-Gitter gibt unter den Äquivalenztransformationen. Durch Benutzung der Minlex-Form für die Repräsentanten haben wir dadurch für die Repräsentanten die beiden Shidoku-Gitter:

In [ ]:
shidoku_1 = np.array([[1,2,3,4],[3,4,1,2],[2,1,4,3],[4,3,2,1]])
shidoku_2 = np.array([[1,2,3,4],[3,4,1,2],[2,3,4,1],[4,1,2,3]])

print("Typ-1 representant:")
print(shidoku_1)
print("-----------")
print("Typ-2 representant:")
print(shidoku_2)

Wir wollen nun erneut numerisch beweisen, dass genau zwei Äquivalenzklassen existieren und die eben definierten Shidoku-Gitter Repräsentanten dieser Äquivalenzklassen sind. Dafür erzeugen wir mit unserer Funktion alle equivalenten Shidoku-Gitter der Repräsentanten.

In [ ]:
type_one_equi = equivalent_grids(shidoku_1)
print("The count of the equivalence class of the Type-1 Shidoku is:", len(type_one_equi))

type_two_equi = equivalent_grids(shidoku_2)
print("The count of the equivalence class of the Type-2 Shidoku is:", len(type_two_equi))

Die numerische Berechnung zeigt also erneut, dass die Äquivalenzklassen des Typ-1 Shidoku-Gitters $96$ Elemente enthält. während die Äquivalenzklasse des Typ-2 Shidoku-Gitters $192$ Elemente enthält. Dadurch haben wir folgende Aussagen:
1. Das Typ-1 Shidoku-Gitter kann nicht äquivalent sein zu dem Typ-2 Shidoku-Gitter, da sie in verschiedenen Äquivalenzklassen liegen.
2. Es existiert keine weiter Äquivalenzklasse, da alle Gitter entweder in der Äquivalenzklasse eines der beiden Gitter ist. Dies folgt direkt aus der Anzahl der vollständigen Gitter: $288 = 96 + 192$.

Diese beiden Gitter sind also gültige Repräsentanten.\
Wir wollen nun nützliche Blaupausen suchen. Wir werden sehen, dass wir tatsächlich nur eine nützliche $1\times 2$ Blaupause benötigen, um auf einen Widerspruch zu kommen. Diese führt uns zu der $1 \times 2$ Blaupause:

In [ ]:
blueprint = np.array([1, 0, 2, 0, 2, 0, 1, 0]) # saved as an array for the pattern checking later
print("Die einzige nützliche Blaupause:")
print(np.reshape(blueprint, (2,4)))

Nun müssen wir eine Routine programmieren, welche die Blaupause gegen die möglichen Permutationen des Gitters prüft. Für die Reihenpermutationen müssen wir zwei Möglichkeiten betrachten, es muss jeweils das Band betrachtet werden. Eine Reihenpermutation innerhalb der Bänder ist nicht nötig, da diese die Struktur der Blaupause nicht verändern und somit keinen Unterschied machen würden. Wir haben also die Reihenpermutationen:
\begin{align*}
    &\sigma_1 = (0,1,2,3) \mapsto (0,1,2,3)\\
    &\sigma_2 = (0,1,2,3) \mapsto (2,3,0,1)
\end{align*}
Für die Spaltenpermutationen benötigen wir vier Permutationen, dabei existieren mehrere Möglichkeiten diese auszudrücken. Eine Möglichkeit hierfür wäre in jedem Band die möglichen Permutationen und die Bänder nicht zu permutieren. Dies liefert die Spaltenpermutationen:
\begin{align*}
    &\rho_1 = (0,1,2,3) \mapsto (0,1,2,3)\\
    &\rho_2 = (0,1,2,3) \mapsto (1,0,2,3)\\
    &\rho_3 = (0,1,2,3) \mapsto (0,1,3,2)\\
    &\rho_4 = (0,1,2,3) \mapsto (1,0,3,2)
\end{align*}
Da wir die Blaupause wieder in der Größe $m\times n$ mit $m\leq n$ eingeschränkt hatten, müssen wir die Transposition jeweils auch noch betrachten.\
Insgesamt müssen also $2 \cdot 4 \cdot 2 = 16$ äquivalente Gitter erzeugt werden und gegen die Blaupause verglichen werden. Nach der Folgerung aus Bemerkung 3.19 muss das erzeugte äquivalente Gitter nur gegen Blaupausen überprüft werden, falls die Zellen $z_{1,1}$ und $z_{2,3}$ die gleiche Zahl zugeordnet haben.

In [ ]:
def generate_unavoidable(grid, blueprint):
    """
    This function calculates all unavoidable sets in the given grid that have the form of the given blueprint.
    """
    
    unavoid_sets = [] # list for the found unavoidable sets
    # check which numbers are contained in the blueprint and in which cell the first occurance is
    numbers, first_occurances = np.unique(blueprint, return_index = True)[:2]

    for i in transpose:
        for j in ([0,1,2,3],[2,3,0,1]): # needed row permutations as defined before
            for k in ([0,1,2,3],[1,0,2,3],[0,1,3,2],[1,0,3,2]): # needed column permutations as defined before
                equiv_grid = permutate_columns(permutate_rows(transpose_grid(grid,i),j),k) # generate equivalent grid
                
                if equiv_grid[0][0] == equiv_grid[1][2]: # test if cells z_{1,1} and z_{2,3} contain the same number
                    equiv_grid = np.reshape(equiv_grid, 16) # reshape grid for blueprint machting
                    unavoid_set = np.zeros(16)
                    
                    for l in range(1,len(numbers)): # exclude the zeros (empty cells) 
                        number = numbers[l]
                        number_grid = equiv_grid[first_occurances[l]] # the number contained by the equivalent grid
                        
                        # check if the cells of the blueprint containing a specific number overlap with the cells of the 
                        # equivalent grid containing a (maybe other) specific number.
                        if np.array_equal(np.where(equiv_grid[:len(blueprint)] == number_grid)[0],
                                          np.where(blueprint == number)[0]):
                            # add the cells of the equivalent grid to the unavoidable set
                            np.put(unavoid_set, np.where(equiv_grid[:len(blueprint)] == number_grid)[0], number_grid)
                        
                        # if not, set the unavoidable set zero
                        else:
                            unavoid_set = np.zeros(16)
                    
                    # check if a unavoidable set was found
                    if not np.array_equal(unavoid_set, np.zeros(16)):
                        unavoid_set = np.reshape(unavoid_set, (4,4)) # change back the form of the unavoidable set
                        # transpose the unavoidable set back to fit the original grid
                        unavoid_set = transpose_grid(permutate_rows(permutate_columns(unavoid_set,k),j),i)
                        unavoid_sets.append(unavoid_set)
                        
    return unavoid_sets

Mit dieser Methode können wir nun also für ein gegebenes vollständiges Gitter alle unvermeidbaren Mengen mit der Form der Blaupause bestimmen. Dies wollen wir nun auf unsere Repräsentanten anwenden. Wir erzeugen nun zuerst die unvermeidbaren Mengen für das Typ-1 Shidoku-Gitter.

In [ ]:
unavoid_sets_1 = generate_unavoidable(shidoku_1,blueprint)

for sets in unavoid_sets_1:
    print(sets)
    print("---------------")

Wir sehen hierbei, dass die acht gefundenen unvermeidbaren Mengen zwei disjunkte Zerlegungen des Typ-1 Shidoku-Gitters bilden (die ersten $4$ unvermeidbaren Mengen bilden eine disjunkte Zerlegung und die anderen $4$ unvermeidbaren Mengen ebenso). Da alle diese unvermeidbaren Mengen $(4,1)$-unvermeidbar sind, bildet also das Typ-1 Shidoku-Gitter durch jede der beiden disjunkten Zerlegungen jeweils selbst eine $(16,4)$-unvermeidbare Menge nach Korollar 3.26. Nach Proposition 3.29 müssen dann aus dem Typ-1 Gitter mindestens $4$ Tipps bezogen werden, dass ein gültiges Shidoku-Rätsel entstehen kann. Somit kann in dem Typ-1 Shidoku-Gitter kein $3$-Tipp Rätsel existieren.\
Nun wollen wir gleich das Typ-2 Shidoku-Gitter betrachten. Wir erzeugen nun ebenfalls die unvermeidbaren Mengen für dieses Gitter.

In [ ]:
unavoid_sets_2 = generate_unavoidable(shidoku_2, blueprint)
for sets in unavoid_sets_2:
    print(sets)
    print("---------------")

Wir sehen erneut, dass die vier gefundenen unvermeidbaren Mengen eine disjunkte Zerlegung des Typ-2 Shidoku-Gitters bilden. Diese unvermeidbaren Mengen sind alle wieder $(4,1)$-unvermeidbar, womit das Typ-2 Shidoku-Gitter wieder selbst eine $(16,4)$-unvermeidbare Menge ist. Somit müssten erneut mindestens $4$ Tipps aus dem Gitter bezogen werden, damit ein gültiges Shidoku-Rätsel entstehen kann. Somit kann auch in dem Typ-2 Shidoku-Gitter kein $3$-Tipp Rätsel existieren.\
Durch die Invarianz der Existenz eines $3$-Tipp Rätsel unter den Äquivalenzklassen können wir also insgesamt mit Sicherheit sagen:
<h2><center> Es existiert kein $3$-Tipp Shidoku! </center></h2>